In [ ]:
import sys, os
from datetime import datetime
# Setup
skd_python_dir = os.path.dirname(os.getcwd())
if(skd_python_dir not in sys.path):
    sys.path.append(skd_python_dir)

# Import local skd_libraries
import skd_collision_tests.collision_environment.collision_env_utils as collision_env_utils
import skd_collision_tests.collision_environment.collision_environment as collision_environment
import skd_collision_tests.collision_environment.collision_experiments_loader as collision_loader
import skd_collision_tests.collision_environment.collision_data_analyser as collision_data_analyser


# Get a shared timestamtp
timestamp = datetime.now().strftime("%m-%d-%H-%M-%S")
outdir_header = "final_collision_experiments"


### Using the Collision Test Environment
The collision environment allows to quickly prototype and visualize the interactions of an implemented car controller against safe trajectories saved in a json file.
See "/skd_trajectories/skd_trajectories_db/" for examples of the format.


### Usage:
To use the module, only the following 3 steps are needed:
- Generate a collision experiments configuration file
- Execute experiments according to configuration file
- Parse and analyse the data from the collision experiments' summary file


### Generate a config file, given an existing safe trajectory file in json

In [ ]:
# Example function to generate collision_exp config
def gen_collision_exp_config(output_dir):
    output_path = output_dir + "/collision_config.yaml"
    
    # Set optional configurations
    # Safe trajectory files to consider, if empty, then GUI will allow user to select
    safe_files = []
    # Controller ids in experiment
    multiplier_ids=[0.5, 0.625]
    # Number of collision experiments per safe trajectory
    num_runs=50
    # Maximum number of steps simulated per experiment
    max_num_steps=25
    # Maximum number of trajectories considered per safe trajectory file
    max_trajs_per_file=-1
    # Default controller behaviour type ("basic" only for now)
    car_controller_type="basic"
    
    collision_env_utils.gen_collision_experiments_config(output_path, safe_files, num_runs, max_num_steps, max_trajs_per_file,
                                car_controller_type, multiplier_ids)
    return output_path




# Generate the configuration file
outputdir = os.getcwd() + "/testing_output/%s_%s" % (outdir_header, timestamp) 

# Create unique output dir
try:
    os.makedirs(outputdir)
except OSError as error:
    print(error)

config_filepath = gen_collision_exp_config(outputdir)

### Executing collision experiments from config file

In [ ]:
#### Executing collision experiments from config file

def run_collision_env(output_dir, config_file):
    # Create a loader for the config file
    experiments_loader = collision_loader.CollisionExperimentLoader(output_dir, config_file)

    # Run experiments from the config file
    experiments_loader.run_collision_experiments()

    # Get collision summary stats
    summary_filepath = experiments_loader.get_summary_file_path()

    return summary_filepath


# Execute experiments from config
experiments_outdir = outputdir +  "/controllers" 
summary_outpath = run_collision_env(experiments_outdir, config_filepath)



### Analyse data from collision experiments from summary file

In [ ]:
def get_collision_summary_stats(output_dir, summary_stats_file):

    """ Create a collision experiment analyser """
    collision_analyser = collision_data_analyser.CollisionExperimentDataAnalyser(summary_stats_file, output_dir)

    """ Parse the data and review it on output_dir """
    collision_analyser.get_analyzer_summary_statistics()
    

    
# Parse results
results_outdir = outputdir + "/collision_experiments_data"

try:
    os.makedirs(results_outdir)
except OSError as error:
    print(error)

get_collision_summary_stats(results_outdir, summary_outpath)
print("DONE")
